# Load Packages

In [2]:
import os, datetime, glob, re, json, requests, time
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import googlemaps
import numpy as np
import pandas as pd
import geopandas as gpd
from pandas import json_normalize

import warnings
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore")

/Users/tim.cho/opt/anaconda3/envs/mac_k2/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


# Data Clean

## 捐血中心位置

In [3]:
# 檔案名稱：poi_clean_cht.csv 
# 臺北市捐血中心固定捐血點，政府公開資料：https://data.gov.tw/dataset/132446
path = r"/Users/tim.cho/Documents/project_me/y2024_tppass/臺北市捐血中心固定捐血點_捐血室.csv"
df_捐血中心 = pd.read_csv(path, encoding='big5')
df_捐血中心.head()

,捐血室名稱,行政區,設站地址
0,關渡捐血室,63000120,台北市北投區立德路123號
1,市府捐血室,63000020,台北市信義區市府路1號
2,長春捐血室,63000010,台北市松山區復興北路69號5樓
3,南海捐血室,63000050,台北市中正區南海路1號3樓
4,公園號捐血車,63000050,台北市中正區襄陽路2號


## POI data

In [36]:
# 檔案名稱：poi_clean_cht.csv 
# 目的：從中華電信商家黃頁爬蟲下載的 商家地標，用來模擬未來可能潛在合作的商家
# 這份資料後續實際應用上，需要由市府與願意合作的商家，陸續去洽談後才是真實資料，屬於地推商業的部分。
path = r"/Users/tim.cho/Documents/project_me/y2024_tppass/poi_clean_cht.csv"
df_chtpoi = pd.read_csv(path)
df_chtpoi.head()

,商店名稱,類別1,類別2,地址,lon,lat,POI類別
0,磨豆花棧,美食佳餚,冰品店,NaN,121.568582,24.987011,吃的
1,品勝企業社,美食佳餚,冰品店,NaN,121.616739,25.054247,吃的
2,三一冰淇淋股份有限公司,美食佳餚,冰品店,NaN,121.517058,25.047176,吃的
3,三一冰淇淋股份有限公司,美食佳餚,冰品店,NaN,121.563904,25.034026,吃的
4,中敦國際股份有限公司,美食佳餚,冰品店,NaN,121.568380,25.043996,吃的


## 爬蟲PTT

In [37]:
# PTT 網頁網址，我隨意找尋 近年 3-4個月由 PTT網民 彙整的活動
url = [
    'https://www.ptt.cc/bbs/Lifeismoney/M.1725495478.A.D8F.html',
    'https://www.ptt.cc/bbs/Lifeismoney/M.1724482650.A.EB1.html',
    'https://www.ptt.cc/bbs/Lifeismoney/M.1721460420.A.CBD.html',
    'https://www.ptt.cc/bbs/Lifeismoney/M.1717436215.A.65A.html']

In [38]:
list_all_dfs=[]
for url_i in url:
    try:
        response = requests.get(url_i)
        # 使用 BeautifulSoup 解析 HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        article_content = soup.find(id="main-content").text
        article_content_no_spaces = re.sub(r'\s+', '', article_content)
        sections = re.split(r'(地區)', article_content_no_spaces)

        # 讀取這個網頁的 data，先分成兩大欄位 之後從 內容分別擷取需要的
        data = []
        for i in range(1, len(sections), 2): 
            area = sections[i]
            content = sections[i + 1]
            data.append({'區域': area, '內容': content})
        df = pd.DataFrame(data)

        # 迴圈分一個個地區分析
        list_dfs = []
        for i in range(0, len(df['內容'])-2):
            try:
                content_t = df['內容'][i]
                segments = re.split(r'(\d{1,2}/\d{1,2}\(.+?\)：)', content_t)
                #print(f"成功階段:{url_i}")
                data = []
                for i in range(1, len(segments), 2):  # 跳過非日期的部分
                    date = segments[i]
                    content = segments[i + 1] if i + 1 < len(segments) else ""
                    data.append({'日期': date, '內容': content.strip()})
                df_one_area = pd.DataFrame(data)
                # 先將內容中的「▏▎▍▋▊▉」這些字元 後面的部分 都去除
                df_one_area['內容'] = df_one_area['內容'].str.replace(r'[▏▎▍▋▊▉].*', '', regex=True)
                df_one_area['日期'] = df_one_area['日期'].str.replace("：", "")
                df_one_area[['日期', '星期']] = df_one_area['日期'].str.extract(r'(\d+/\d+)\((.)\)')
                df_one_area['優惠內容_ori'] = df_one_area['內容'].str.split("https:").str[0]
                df_one_area[['捐血地點', '營業時間', '優惠內容']] = df_one_area['優惠內容_ori']\
                                                                .str.extract(r'(.+?)(\d{2}:\d{2}~\d{2}:\d{2})(.*)?')
                df_one_area['圖片網址'] = "https:" + df_one_area['內容'].str.split("https:").str[1]
                del df_one_area['內容'], df_one_area['優惠內容_ori']
                list_dfs.append(df_one_area)
            except:
                pass
        df_地點與活動 = pd.concat(list_dfs).reset_index(drop=True)
        df_地點與活動['日期'] = "2024/" + df_地點與活動['日期']
        df_地點與活動['日期'] = pd.to_datetime(df_地點與活動['日期'])
        list_all_dfs.append(df_地點與活動)
        print(f"  成功_抓取:{url_i}")
    except:
        print(f"!!!!失敗的:{url_i}")

  成功_抓取:https://www.ptt.cc/bbs/Lifeismoney/M.1725495478.A.D8F.html
  成功_抓取:https://www.ptt.cc/bbs/Lifeismoney/M.1724482650.A.EB1.html
  成功_抓取:https://www.ptt.cc/bbs/Lifeismoney/M.1721460420.A.CBD.html
  成功_抓取:https://www.ptt.cc/bbs/Lifeismoney/M.1717436215.A.65A.html


In [39]:
# 基本上可以清出 大多數有用的欄位
df_地點與活動_全部 = pd.concat(list_all_dfs)
df_地點與活動_全部.head()

,日期,星期,捐血地點,營業時間,優惠內容,圖片網址
0,2024-09-20,五,桃園內壢家樂福,09:30~16:30,送家樂福禮券$300/$600,https://i.imgur.com/MeqyHSD.jpeg
1,2024-09-22,日,台南中西區郡緯街和勝堂牌樓前,09:30~17:30,送全聯禮券$250/$600等好禮,https://i.imgur.com/6PTpD03.jpeg
2,2024-09-29,日,雲林虎尾圖書館,09:00~16:00,送全聯、米、醬油禮盒等，還有抽獎1次/2次#前百名加贈鴨肉羹,https://i.imgur.com/XCzh6ET.jpeg《長期活動》雲林捐血站捐血加...
3,2024-10-06,日,雲林虎尾國小,08:30~16:00,送米、蛋、醬油禮盒等好禮/加倍送#現場有豐富公益活動,https://i.imgur.com/qDcpVnN.jpeg
4,2024-11-17,日,台南開山路延平郡王祠前,09:00~16:00,送全聯禮券$200/加倍送，皆有小書包,https://i.imgur.com/IJsc90x.jpeg


# Gmap api 加上經緯度

## def 定義函數

In [41]:
# 定義地址 簡單清理的函數  #後來發現不用用到也可以
# def convert_number_to_chinese(num_str):
#     num_dict = {'0': '零', '1': '一', '2': '二', '3': '三', '4': '四', '5': '五',
#                 '6': '六', '7': '七', '8': '八', '9': '九'}
#     chinese_num = ''.join([num_dict[char] for char in num_str])
#     return chinese_num
# def replace_num2ch(address):
#     pattern = r'(\d+)段'
#     def replace_match(match):
#         num_str = match.group(1)
#         chinese_num = convert_number_to_chinese(num_str)
#         return chinese_num + '段'
#     new_address = re.sub(pattern, replace_match, address)
#     return new_address

# 地址欄位 除了標準地址以外，也適用類似「地標」名稱的
# 記得替換成自己 google map api_key 'AAAAA11111'!!!!!!!!!!!!!!!!!
API_KEY = 'AAAAA11111'
def catch_coord(address_t):
    # 將地標名稱或地址中的空格轉換為 '+'
    params = {'key': API_KEY, 'address': address_t.replace(' ', '+'), 'language': 'zh-TW'}
    # Geocoding API 的基本 URL
    base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
    response = requests.get(base_url, params=params)
    data = response.json()
    if data['status'] == 'OK':
        result = data['results'][0]
        location = result['geometry']['location']
        g_address = data['results'][0]['formatted_address']
        list_item_t = [address_t, g_address, location['lat'], location['lng']]
        return list_item_t
    else:
        # 當地標無法找到時，返回預設值
        list_item_t = [address_t, '-', 0, 0]
        return list_item_t

## 執行加經緯度 func

In [46]:
# 準備清理，address_t 就是我要置入的欄位名稱
df_miss = df_地點與活動_全部.copy()
address_t = "捐血地點"
df_miss.head(3)

,日期,星期,捐血地點,營業時間,優惠內容,圖片網址
0,2024-09-20,五,桃園內壢家樂福,09:30~16:30,送家樂福禮券$300/$600,https://i.imgur.com/MeqyHSD.jpeg
1,2024-09-22,日,台南中西區郡緯街和勝堂牌樓前,09:30~17:30,送全聯禮券$250/$600等好禮,https://i.imgur.com/6PTpD03.jpeg
2,2024-09-29,日,雲林虎尾圖書館,09:00~16:00,送全聯、米、醬油禮盒等，還有抽獎1次/2次#前百名加贈鴨肉羹,https://i.imgur.com/XCzh6ET.jpeg《長期活動》雲林捐血站捐血加...


In [ ]:
# 用這一串 取出地址變數並迴圈執行
list_gmap_use = list(set(df_miss[f'{address_t}']))
list_dfs=[]
for poi_sp in list_gmap_use:
    try:
        list_t = catch_coord(poi_sp)
        list_dfs.append(list_t)
        time.sleep(0.3)
    except:
        list_t = [poi_sp, "-", 0, 0]
        list_dfs.append(list_t)
        time.sleep(0.3)
        print(f"{poi_sp} 失敗了！")
        
df_gmap_result = pd.DataFrame(list_dfs)
df_gmap_result.columns = [f'{address_t}','g地址','lat','lon']

In [ ]:
df_miss_m = df_miss.merge(df_gmap_result, on=[f'{address_t}'])\
                   .reset_index(drop=True)
df_miss_m.head()

In [253]:
## 用 gmap api 轉換捐血地點的經緯度地址的成果
df_地點與活動_gmap = df_miss_m.copy()
df_地點與活動_gmap.head()

,日期,星期,捐血地點,營業時間,優惠內容,圖片網址,g地址,lat,lon
0,2024-09-06,五,新北板橋車站北二門,09:30~16:30,送全聯禮券$300/$600,https://i.imgur.com/KIHoXBz.jpeg,220台灣新北市板橋區縣民大道二段7號板橋車站,25.014050,121.463810
1,2024-09-08,日,新北泰山文程廣場,09:00~16:30,送米、蛋、熱敷眼罩.衛生紙/加倍送#現場有免費血糖檢測、義剪、精油按摩活動,https://i.imgur.com/TiygrRQ.jpeg,243台灣新北市泰山區明志路二段194號文程廣場,25.050413,121.427898
2,2024-09-08,日,北市二二八公園,10:00~18:00,,NaN,100台灣台北市中正區凱達格蘭大道3號二二八和平公園,25.040298,121.515545
3,2024-11-17,日,新北中和中安街圖書館旁,09:00~17:00,送牛排,https://i.imgur.com/UyHtda0.jpeg,235台灣新北市中和區中安街85號,25.001429,121.512703
4,2024-09-09,一,新北汐止派出所,09:30~16:30,送牛排、蛋、三合一充電線/加倍送,https://i.imgur.com/04gl848.jpeg,221台灣新北市汐止區大同路二段459號,25.068018,121.659948


In [255]:
# 捐血活動位置
path_out = r"/Users/tim.cho/Documents/project_me/y2024_tppass/捐血活動優惠_最新月份.csv"
df_地點與活動_gmap.to_csv(path_out, index=False)